# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("C:/Users/Aline/anaconda3/CSV Files/WeatherPy/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Egilsstadir,65.2653,-14.3948,-6.05,93,0,3.09,IS,1699669353
1,1,Mogocha,53.7333,119.7667,-9.64,47,76,1.48,RU,1699669353
2,2,Albany,42.6001,-73.9662,7.04,61,50,2.41,US,1699669353
3,3,Grytviken,-54.2811,-36.5092,2.00,85,53,5.77,GS,1699669353
4,4,Puerto Natales,-51.7236,-72.4875,4.78,96,100,2.82,CL,1699669353


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [63]:
%%capture --no-display
#Transform latitude and longitude to float
city_data_df["Lat"] = city_data_df["Lat"].astype(float)
city_data_df["Lng"] = city_data_df["Lng"].astype(float)

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
)

# Display the map
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [28]:
# Narrow down cities that fit criteria and drop any results with null values
# A max temperature lower than 24 degrees but higher than 15.5.
# Wind speed less than 10 mph.
# Cloudiness less than 40
# Humidity between 35% and 70%.
ideal_city_df = city_data_df.loc[(city_data_df["Max Temp"]>15.5) & 
                                 (city_data_df["Max Temp"]<24) &
                                 (city_data_df["Wind Speed"]<10) &
                                 (city_data_df["Cloudiness"]<40) &
                                 (city_data_df["Humidity"]>35) & 
                                 (city_data_df["Humidity"]<70)]

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
35,35,Carnarvon,-24.8667,113.6333,23.27,56,2,8.03,AU,1699669359
55,55,Awjilah,29.1081,21.2869,19.69,54,0,2.75,LY,1699669362
83,83,Pacific Grove,36.6177,-121.9166,16.25,53,0,2.06,US,1699669366
93,93,Puerto Peñasco,31.3167,-113.5333,20.51,38,0,1.34,MX,1699669369
105,105,Papatowai,-46.5619,169.4708,18.46,41,0,1.89,NZ,1699669371
109,109,Rio Pomba,-21.2747,-43.1792,21.75,69,3,0.76,BR,1699669372
111,111,Codrington,-38.2667,141.9667,16.33,50,7,7.09,AU,1699669372
149,149,Sirte,31.2089,16.5887,21.60,43,0,4.14,LY,1699669377
156,156,Mount Gambier,-37.8333,140.7667,17.42,43,6,5.09,AU,1699669378
164,164,Bayjī,34.9311,43.4870,19.20,36,38,2.89,IQ,1699669379


### Step 3: Create a new DataFrame called `hotel_df`.

In [34]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df.drop(["City_ID", "Max Temp", "Cloudiness", "Wind Speed", "Date"], axis=1, inplace=True)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
0,Egilsstadir,65.2653,-14.3948,93,IS,
1,Mogocha,53.7333,119.7667,47,RU,
2,Albany,42.6001,-73.9662,61,US,
3,Grytviken,-54.2811,-36.5092,85,GS,
4,Puerto Natales,-51.7236,-72.4875,96,CL,
...,...,...,...,...,...,...
543,Center,31.5107,-96.4247,77,US,
544,Hatillo,18.4863,-66.8254,92,PR,
545,Alaghsas,17.0187,8.0168,17,NE,
546,Hūn,29.1268,15.9477,35,LY,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [62]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "categories":"accommodation.hotel",
    "limit": 10,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lng = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Egilsstadir - nearest hotel: Valaskjálf
Mogocha - nearest hotel: 7Ka
Albany - nearest hotel: No hotel found
Grytviken - nearest hotel: No hotel found
Puerto Natales - nearest hotel: Dorotea Patagonia Hostel
Praia da Vitória - nearest hotel: Salles
Olonkinbyen - nearest hotel: No hotel found
Goz Beida - nearest hotel: No hotel found
Mapastepec - nearest hotel: Hotel San Diego
Carazinho - nearest hotel: San Remo Hotel
Coahuayana de Hidalgo - nearest hotel: No hotel found
Gülnar - nearest hotel: No hotel found
Blackmans Bay - nearest hotel: Villa Howden
Afaahiti - nearest hotel: Omati Lodge
George Town - nearest hotel: Page 63 hostel
Richards Bay - nearest hotel: Bon Hotel Waterfront
Alofi - nearest hotel: Matavai Resort
Rémire-Montjoly - nearest hotel: Complexe Belova
Tura - nearest hotel: No hotel found
Waitangi - nearest hotel: Hotel Chathams
Āsosa - nearest hotel: Africa Hotel
Gamba - nearest hotel: No hotel found
Champotón - nearest hotel: Paraíso Beach
Stanley 

,City,Lat,Lng,Humidity,Country,Hotel Name
0,Egilsstadir,65.2653,-14.3948,93,IS,Valaskjálf
1,Mogocha,53.7333,119.7667,47,RU,7Ka
2,Albany,42.6001,-73.9662,61,US,No hotel found
3,Grytviken,-54.2811,-36.5092,85,GS,No hotel found
4,Puerto Natales,-51.7236,-72.4875,96,CL,Dorotea Patagonia Hostel
...,...,...,...,...,...,...
543,Center,31.5107,-96.4247,77,US,No hotel found
544,Hatillo,18.4863,-66.8254,92,PR,No hotel found
545,Alaghsas,17.0187,8.0168,17,NE,Hôtel Zagado
546,Hūn,29.1268,15.9477,35,LY,فندق الهروج


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [66]:
%%capture --no-display

# Configure the map plot
city_hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriImagery",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
city_hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)